In [2]:
import psycopg2
import snowflake.connector
import sqlite3
import pandas as pd

# Configurações do PostgreSQL
pg_conn = psycopg2.connect(
    host="000.00.000.0",
    database="Sua_base",
    user="user",
    password="senha"
)

# Conexão SQLite -- Banco interno para parametros - (criação do banco na pasta database)
conn = sqlite3.connect('C:/ProjetosPython/EDD-DL/Gerar_Script_Sql/database/config_scripts_sql.db')
cursor = conn.cursor()

def column_add(columns, column_name, column_type):
    # Adicione a nova coluna à lista de colunas
    columns.append((column_name, column_type))

def get_pg_table_structure(table_name):
    with pg_conn.cursor() as cursor:
        cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table_name}'")
        return cursor.fetchall()

#add colunas de controle 
def get_pg_table_structure_with_added_column(table_name):
    columns = get_pg_table_structure(table_name)
    column_add(columns, 'CREATED_AT', 'character varying')
    column_add(columns, 'UPDATED_AT', 'character varying')
    column_add(columns, 'ID_DATABASE', 'character varying')
    return columns

def create_sf_table(table_name, columns):
    create_query = f"CREATE OR REPLACE TABLE {table_name} ({', '.join([f'{col[0]} {col[1]}' for col in columns])})"
    print(create_query)

def migrate_table(pg_table_name, sf_table_name):
    pg_columns_with_added = get_pg_table_structure_with_added_column(pg_table_name)
    create_sf_table(sf_table_name, pg_columns_with_added)

try:
    # Leitura das tabelas do SQLite
    query_tabelas_full = "SELECT * FROM config_tabelas_full WHERE ATIVO = 'true';"
    df_tabelas_full = pd.read_sql_query(query_tabelas_full, conn)

    for _, tabelas_full in df_tabelas_full.iterrows():
        SCHEMA = tabelas_full['SCHEMA']
        TABELA = tabelas_full['TABELA']
        
        # Exemplo de migração de uma tabela com a nova coluna 'CREATED_AT'
        migrate_table(TABELA, f"RESSARCIMENTO.CURATED_DATA.{TABELA}")
        
except Exception as err:
    print(err)

NameError: name 'sqlite3' is not defined